## Causal Inference

In [2]:
import pandas as pd


In [4]:
df = pd.read_json('/workspaces/chat-gpt-failures/data/causal_prompting_pd_T2_normalized.json')

In [5]:
df.head()

,y_levenshtein,t_binary_treatment,t_treatment,z_prompt_size,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
0,0.347670,False,control,16,0,9,101,1,12,48,71
1,0.554165,False,control,175,0,17,417,13,100,240,375
2,0.017834,False,control,9,0,13,208,3,17,153,253
3,0.009490,False,control,50,0,12,154,1,14,137,249
4,0.358811,False,control,38,0,13,127,3,21,82,137


## I. Create a causal model from the data and given graph.

In [6]:
from dowhy import CausalModel

In [7]:
df[df["t_treatment"]!="T2"]

,y_levenshtein,t_binary_treatment,t_treatment,z_prompt_size,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
0,0.347670,False,control,16,0,9,101,1,12,48,71
1,0.554165,False,control,175,0,17,417,13,100,240,375
2,0.017834,False,control,9,0,13,208,3,17,153,253
3,0.009490,False,control,50,0,12,154,1,14,137,249
4,0.358811,False,control,38,0,13,127,3,21,82,137
...,...,...,...,...,...,...,...,...,...,...,...
5841,0.151751,True,T1,16,0,11,5,1,44,33,51
5842,0.160284,True,T1,30,0,9,140,2,6,31,118
5843,0.425051,True,T1,19,0,9,51,1,7,39,55
5844,0.652414,True,T1,110,0,17,603,6,33,204,328


In [8]:
model_T1 = CausalModel(
        data = df[df["t_treatment"]!="T2"],
        treatment = ['t_binary_treatment'],
        outcome = ['y_levenshtein'],
        effect_modifiers = ['z_ast_levels','z_n_ast_nodes'],
        common_causes= ['z_prompt_size','z_n_whitespaces','z_token_counts','z_nloc']
    )

In [9]:
model_T2 = CausalModel(
        data = df[df["t_treatment"]!="T1"],
        treatment = ['t_binary_treatment'],
        outcome = ['y_levenshtein'],
        #effect_modifiers = ['z_n_ast_errors','z_ast_levels','z_n_whitespaces','z_complexity','z_nloc','z_token_counts','z_n_ast_nodes'],
        effect_modifiers = ['z_ast_levels','z_n_ast_nodes'],
        common_causes= ['z_prompt_size','z_n_whitespaces','z_token_counts','z_nloc']
    )

## II. Identify causal effect and return target estimands.

Find a math expression to identify de-confounders form confounders

In [10]:
identified_estimand_T1 = model_T1.identify_effect()

In [11]:
identified_estimand_T2 = model_T2.identify_effect()

# III. Estimate the target estimand using a statistical method.


In [12]:
causal_estimate_match_T1 = model_T1.estimate_effect(identified_estimand_T1,method_name="backdoor.propensity_score_weighting") #"backdoor.propensity_score_stratification"

propensity_score_weighting


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[self.propensity_score_column] = self.propensity_score_model.predict_proba(self._observed_common_causes)[:, 1]
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_weighting_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [13]:
causal_estimate_match_T2 = model_T2.estimate_effect(identified_estimand_T2,method_name="backdoor.propensity_score_weighting")

propensity_score_weighting


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[self.propensity_score_column] = self.propensity_score_model.predict_proba(self._observed_common_causes)[:, 1]
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_weighting_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [14]:
print(causal_estimate_match_T1)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                                                                   
─────────────────────(E[y_levenshtein|z_prompt_size,z_nloc,z_n_whitespaces,z_t
d[t_binary_treatment]                                                         

             
oken_counts])
             
Estimand assumption 1, Unconfoundedness: If U→{t_binary_treatment} and U→y_levenshtein then P(y_levenshtein|t_binary_treatment,z_prompt_size,z_nloc,z_n_whitespaces,z_token_counts,U) = P(y_levenshtein|t_binary_treatment,z_prompt_size,z_nloc,z_n_whitespaces,z_token_counts)

## Realized estimand
b: y_levenshtein~t_binary_treatment+z_prompt_size+z_nloc+z_n_whitespaces+z_token_counts
Target units: ate

## Estimate
Mean value: -0.0517670634208689



In [15]:
print(causal_estimate_match_T2)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                                                                   
─────────────────────(E[y_levenshtein|z_prompt_size,z_nloc,z_n_whitespaces,z_t
d[t_binary_treatment]                                                         

             
oken_counts])
             
Estimand assumption 1, Unconfoundedness: If U→{t_binary_treatment} and U→y_levenshtein then P(y_levenshtein|t_binary_treatment,z_prompt_size,z_nloc,z_n_whitespaces,z_token_counts,U) = P(y_levenshtein|t_binary_treatment,z_prompt_size,z_nloc,z_n_whitespaces,z_token_counts)

## Realized estimand
b: y_levenshtein~t_binary_treatment+z_prompt_size+z_nloc+z_n_whitespaces+z_token_counts
Target units: ate

## Estimate
Mean value: 0.03332748362678761



# IV. Refute the obtained estimate using multiple robustness checks.


In [16]:
es_placebo_T1 = model_T1.refute_estimate(
                identified_estimand_T1,
                causal_estimate_match_T1,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [17]:
es_placebo_T2 = model_T2.refute_estimate(
                identified_estimand_T2,
                causal_estimate_match_T2,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [18]:
print(es_placebo_T1)

Refute: Use a Placebo Treatment
Estimated effect:-0.0517670634208689
New effect:-0.0002474956562447866
p value:1.0



In [19]:
print(es_placebo_T2)

Refute: Use a Placebo Treatment
Estimated effect:0.03332748362678761
New effect:-0.00075055292610337
p value:0.8400000000000001



In [20]:

refute_common_cause_T1 = model_T1.refute_estimate(identified_estimand_T1, causal_estimate_match_T1,
                                       method_name="random_common_cause")

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [21]:
refute_common_cause_T2 = model_T2.refute_estimate(identified_estimand_T2, causal_estimate_match_T2,
                                       method_name="random_common_cause")

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [22]:
print(refute_common_cause_T1)

Refute: Add a random common cause
Estimated effect:-0.0517670634208689
New effect:-0.05176153995735795
p value:0.92



In [23]:
print(refute_common_cause_T2)

Refute: Add a random common cause
Estimated effect:0.03332748362678761
New effect:0.03331983408190196
p value:0.8600000000000001



In [24]:
res_subset_T1 =model_T1.refute_estimate(identified_estimand_T1, causal_estimate_match_T1,
        method_name="data_subset_refuter", subset_fraction=0.9)


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [25]:
res_subset_T2 =model_T2.refute_estimate(identified_estimand_T2, causal_estimate_match_T2,
        method_name="data_subset_refuter", subset_fraction=0.9)

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [26]:
print(res_subset_T1)

Refute: Use a subset of data
Estimated effect:-0.0517670634208689
New effect:-0.05176069638729389
p value:1.0



In [27]:
print(res_subset_T2)

Refute: Use a subset of data
Estimated effect:0.03332748362678761
New effect:0.03378656302275167
p value:0.82



## Library Test

In [28]:
import dowhy.datasets 

In [54]:
df_T1.shape

NameError: name 'df_T1' is not defined

In [ ]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5, 
        num_instruments = 0,
        num_treatments=1,
        num_samples=5574,
        treatment_is_binary=True,
        outcome_is_binary=False)
df = data["df"]
df

In [ ]:
causal_model = CausalModel(
        data = df,
        treatment = ['v0'],
        outcome = ['y'],
        common_causes = ['W0','W1','W2','W3','W4']
        #common_causes= ['z_nloc']
    )

In [ ]:
identified_estimand = causal_model.identify_effect()
estimate = causal_model.estimate_effect(
        identified_estimand,
        method_name="backdoor.propensity_score_matching")

In [ ]:
print(estimate)

In [ ]:
es_placebo = causal_model.refute_estimate(
                identified_estimand,
                estimate,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

In [ ]:
print(es_placebo)